In [ ]:
import time

start = time.time()

In [ ]:
import json
import os
import sys
import typing as T

import dotenv
import pandas as pd

dotenv.load_dotenv()

# Secrets
GOOGLE_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
assert GOOGLE_API_KEY is not None, "GOOGLE_PLACES_API_KEY is not set in the `.env` file"
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
assert OPEN_AI_API_KEY is not None, "OPEN_AI_API_KEY is not set in the `.env` file"
MAPBOX_API_KEY = os.getenv("MAPBOX_API_KEY")
assert MAPBOX_API_KEY is not None, "MAPBOX_API_KEY is not set in the `.env` file"

CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, "src")

sys.path.append(SRC_DIR)

In [ ]:
from llm.defs import DAY_COLUMN, ACTIVITY_TYPE_COLUMN, LOCATION_COLUMN

UI_INPUT = False

In [ ]:
inputs = {
    "location": "South Beach Miami, FL",
    "number_of_people": 6,
    "date": "November 2026",
    "duration_days": 3,
    "group_type": "bachelorette party",
    "description": (
        "include boutique hotel options must 4 stars higher onsite spa beach clubs djs"
        "day high end nightclubs list least six restaurant options dinner include least "
        "one nice steakhouse one nice sushi restaurant"
    ),
}

In [ ]:
if UI_INPUT:
    ##############################################################################
    # Skip this block if you want to use the hard coded inputs
    ##############################################################################
    from input_ui import SimpleInputForm

    form = SimpleInputForm()
    form.setup()
    inputs = form.run_form()

In [ ]:
from llm.search import OpenAiSearch
from llm.defs import ITINERARY_PROMPT_TEMPLATE, Itinerary

llm = OpenAiSearch(OPEN_AI_API_KEY)
output = llm.search(inputs, ITINERARY_PROMPT_TEMPLATE, Itinerary)

print(output)

input_tokens, output_tokens, total_tokens = llm.calculate_tokens(inputs, output)
print(f"Input Tokens: {input_tokens}")
print(f"Output Tokens: {output_tokens}")
print(f"Total Tokens: {total_tokens}")

In [ ]:
# Extract the itinerary content from the response
df = pd.DataFrame(output)

print(df)

In [ ]:
# Inputs for the nearby lookup, if store_type is left as None, it will
# default to match the type of the place that we are searching nearby from
MAX_NEARBY_PLACES = 2
radius_meters = 1500

In [ ]:
from google.search import SearchPlaces

search = SearchPlaces(GOOGLE_API_KEY, verbose=True)
itinerary_place_details, nearby_place_details, city_coordinates, total_api_calls = search.search(
    city=inputs["location"],
    itinerary=output,
    radius_meters=radius_meters,
    single_thread=True,
)

In [ ]:
df["place_id"] = [item["id"] for _, item in itinerary_place_details]
print(df)

print(f"Num results: {len(itinerary_place_details)}")
print(f"Total API calls: {total_api_calls}")

In [ ]:
for place, item in itinerary_place_details:
    main_type = item.get("primaryType", item.get("types", ["UNKNOWN"])[0])
    print(f"{item['displayName']['text']}: {main_type}")
    nearby_names = [
        f"{i['displayName']['text']}: {i.get('primaryType', 'UNKNOWN')}"
        for i in nearby_place_details.get(place, [])
    ]
    for name in nearby_names:
        print(f"  - {name}")

with open(os.path.join(ROOT_DIR, "places.json"), "w", encoding="utf-8") as outfile:
    json.dump({"results": itinerary_place_details}, outfile, ensure_ascii=True, indent=4)
with open(os.path.join(ROOT_DIR, "nearby_places.json"), "w", encoding="utf-8") as outfile:
    json.dump(nearby_place_details, outfile, ensure_ascii=True, indent=4)

In [ ]:
total_time = time.time() - start

print(f"Total time taken: {total_time:.2f} seconds")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

assert len(itinerary_place_details) != 0, "No places found in the itinerary"
assert len(nearby_place_details) != 0, "No nearby places found"

# Create a map showing the places in the itinerary and nearby places
print("Creating map...")
px.set_mapbox_access_token(MAPBOX_API_KEY)

places_dfs = {}
color = 0
color_increment = 1.0 / len(itinerary_place_details)
for name, place in itinerary_place_details:
    color += color_increment
    rows = [(place["displayName"]["text"], *place["location"].values(), color)]
    max_clamp = min(MAX_NEARBY_PLACES, len(nearby_place_details.get(name, [])))
    rows.extend(
        [
            (x["displayName"]["text"], *x["location"].values(), color)
            for x in nearby_place_details.get(name, [])[:max_clamp]
        ]
    )
    places_dfs[name] = pd.DataFrame(rows, columns=["name", "latitude", "longitude", "color"])


fig = go.Figure()

for row in df.iterrows():
    places_coords_df = places_dfs[row[1][LOCATION_COLUMN]]
    trace_name = (
        f"Day {row[1][DAY_COLUMN]}: {row[1][ACTIVITY_TYPE_COLUMN]} at {row[1][LOCATION_COLUMN]}"
    )
    fig.add_trace(
        go.Scattermapbox(
            lat=places_coords_df["latitude"],
            lon=places_coords_df["longitude"],
            mode="markers",
            marker=go.scattermapbox.Marker(
                size=5, color=places_coords_df["color"], colorscale="Rainbow", cmin=0, cmax=1
            ),
            text=places_coords_df["name"],
            name=trace_name,
        )
    )

fig.update_layout(
    autosize=True,
    hovermode="closest",
    mapbox=go.layout.Mapbox(
        accesstoken=os.getenv("MAPBOX_API_KEY"),
        bearing=0,
        center=go.layout.mapbox.Center(lat=city_coordinates["lat"], lon=city_coordinates["lng"]),
        pitch=0,
        zoom=12,
    ),
    height=800,
    width=1000,
    title_text=f"Itinerary for `{inputs['group_type']}` in {inputs['location']}",
    title_x=0.5,
    annotations=[
        dict(
            text=(f"Nearby {MAX_NEARBY_PLACES} places are shown for each itinerary location. "),
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.0,
            xanchor="center",
            yanchor="top",
            font=dict(size=14, color="black"),
        )
    ],
)

fig.show()